**NOTE**: to get this working, you must install the `data-analysis` group dependencies using Poetry:

```shell
poetry install --with=data-analysis
```

You should also run this if you are using git:

```shell
poetry run nbstripout --install
```

This will automatically clear output when committing to git :)

We'll create a basic dataframe from `answers.sqlite3`. Then we'll tease out the JSON columns.

In [ ]:
import pandas as pd
import json
import sqlite3

from sklearn.metrics import cohen_kappa_score

from data_analysis_utils import agreement_as_label

Ways to index and group data:

In [ ]:
# by each Java source code file (unit):
BY_UNIT = ["srcml_path", "version"]
# by the PEM variant shown to the rater:
BY_SCENARIO = BY_UNIT + ["variant"]
# by the rater: uniquely identifies one particular data point:
BY_RATER = BY_SCENARIO + ["rater"]

In [ ]:
# Read answers.sqlite3 into a DataFrame
conn = sqlite3.connect("answers.sqlite3")
df = pd.read_sql_query("SELECT * FROM answers", conn)
conn.close()
df

Now time to clean the data — extract it from that JSON column.

Instead of creating a nice schema in `answers.sqlite3`, I decided to defer the job of making nice columns to the data analysis stage. So now we have to parse the answers column as JSON and extract data.


As of 2023-05-02, these are columns:

In [ ]:
df["answers"].apply(json.loads)[0]

In [ ]:
def json_to_columns(df):
    json_column = df["answers"].apply(json.loads)
    return df.assign(
        jargon=json_column.apply(lambda x: x["jargon"]).astype(int),
        sentence_structure=json_column.apply(lambda x: x["sentence_structure"]).astype("category"),
        explanation=json_column.apply(lambda x: x["explanation"]),
        explanation_correctness=json_column.apply(lambda x: x["explanation_correctness"]).astype("category"),
        # I wish I had a better name for this column, but it's basically, "if the explanation is MAYBE correct, WHY is it maybe correct?"
        explanation_maybe=json_column.apply(lambda x: x["explanation_maybe"]).astype("category"),
        fix=json_column.apply(lambda x: x["fix"]).astype("category"),
        fix_correctness=json_column.apply(lambda x: x["fix_correctness"]).astype("category"),
        additional_errors=json_column.apply(lambda x: x["additional_errors"]).astype("category"),
        notes=json_column.apply(lambda x: x["notes"]).astype("string"),
        length=json_column.apply(lambda x: x["length"]).astype(int),
    )

def variant_and_rater_as_categorical(df):
    "variant and rater are both fixed, categorical variables"
    return df.assign(
        variant=df["variant"].astype("category"),
        rater=df["rater"].astype("category"),
    )


def set_empty_notes_to_na(df):
    "Notes that are empty strings should just be missing values"
    return df.assign(notes=df["notes"].replace("", pd.NA))


full_df = df.pipe(variant_and_rater_as_categorical)\
    .pipe(json_to_columns)\
    .pipe(set_empty_notes_to_na)\
    .drop(columns=["answers"])


full_df.sample(5)

In [ ]:
full_df.info()

In [ ]:
assert df["rater"].nunique() == 3
assert df["variant"].nunique() == 4

In [ ]:
# WHY IS THIS 900??!?!?
full_df.groupby(["srcml_path", "version", "variant"]).size()

The length should be the equal regardless of the rater, so let's assert that here:

In [ ]:
def check_lengths(df):
    lengths = df.groupby(BY_SCENARIO).agg({"length": ["min", "max"]})
    assert (lengths["length"]["min"] == lengths["length"]["max"]).all()

# idk why this doesn't work any more 
#check_lengths(full_df)

In [ ]:
two_ratings = full_df.groupby(["srcml_path", "version"]).filter(lambda x: len(x["rater"].unique()) >= 2)
assert len(two_ratings) == len(full_df), "Had fewer than two ratings for some scenarios"

two_ratings

In [ ]:
COLUMNS_TO_RENAME = [
    "jargon",
    "sentence_structure",
    "explanation",
    "explanation_correctness",
    "explanation_maybe",
    "fix",
    "fix_correctness",
    "additional_errors",
    "notes",
    "length",
]


def get_ratings(df, rater):
    return (
        df[df["rater"] == rater]
        .set_index(BY_SCENARIO)
        # We don't need the rater column anymore
        # and earlier we confirmed that the length column is identical
        .drop(columns=["rater", "length"])
        # We have to rename the columns here because the three-way merge will fail to recognize identical columns
        # otherwise:
        .rename(columns={col: f"{col}_{rater}" for col in COLUMNS_TO_RENAME})
    )

# Get the explanation ratings from prajish and eddie:
eddie_ratings = get_ratings(two_ratings, "eddie")
prajish_ratings = get_ratings(two_ratings, "prajish")
brett_ratings = get_ratings(two_ratings, "brett")

assert len(eddie_ratings) == len(prajish_ratings) == len(brett_ratings)
assert len(eddie_ratings) == 40

In [ ]:
ratings = prajish_ratings.join([eddie_ratings, brett_ratings], how="outer")

ratings.sample(3)

In [ ]:
ratings.info()

In [ ]:
eddie_and_prajish_agreement = ratings[ratings["explanation_eddie"].notna() & ratings["explanation_prajish"].notna()]
eddie_and_prajish_agreement

In [ ]:
# TODO: perhaps Scott's pi is a better measure of agreement for this data:
# See: https://stats.stackexchange.com/a/525640

eddie_and_prajish_agreement = ratings[ratings["explanation_eddie"].notna() & ratings["explanation_prajish"].notna()]

kappa = cohen_kappa_score(eddie_and_prajish_agreement["explanation_eddie"].astype(bool), eddie_and_prajish_agreement["explanation_prajish"].astype(bool))
kappa, agreement_as_label(kappa)

In [ ]:
def group_two_raters(rater1, rater2):
    "Returns a DataFrame with only the rows where both raters have rated the scenario"
    return ratings[ratings[f"explanation_{rater1}"].notna() & ratings[f"explanation_{rater2}"].notna()]

def kappa_for_two_raters(column, rater1, rater2):
    "Returns agreement for two raters on a certain column"
    agreement = group_two_raters(rater1, rater2)
    return cohen_kappa_score(agreement[f"{column}_{rater1}"].astype("category"), agreement[f"{column}_{rater2}"].astype("category"))

def kappa_for_two_raters_with_label(column, rater1, rater2):
    kappa = kappa_for_two_raters(column, rater1, rater2)
    return kappa, agreement_as_label(kappa)

In [ ]:
kappa_for_two_raters_with_label("explanation", "eddie", "prajish")

In [ ]:
kappa_for_two_raters_with_label("explanation", "eddie", "brett")

In [ ]:
kappa_for_two_raters_with_label("explanation", "prajish", "brett")

In [ ]:
kappa_for_two_raters_with_label("fix", "eddie", "prajish")

In [ ]:
kappa_for_two_raters_with_label("fix", "eddie", "brett")

In [ ]:
kappa_for_two_raters_with_label("fix", "prajish", "brett")

In [ ]:
eddie = ratings["explanation_eddie"]
brett = ratings["explanation_brett"]
prajish = ratings["explanation_prajish"]

rater1 = prajish.combine_first(brett)
rater2 = eddie.combine_first(brett)


In [ ]:
kappa = cohen_kappa_score(rater1.astype("category"), rater2.astype("category"))
kappa, agreement_as_label(kappa)

Create a summary that is (somewhat) easy to display in Excel:

In [ ]:
full_df\
    .assign(explanation=full_df["explanation"].astype(int))\
    .pivot(index=["srcml_path", "version", "variant"], columns="rater", values=["explanation", "fix", "notes"]).to_excel("full_df.xlsx")